In [1]:
import pandas as pd
from functools import reduce
from sqlalchemy import create_engine

In [2]:
# Read OECD education statistics csv file
csv_file = "Resources/OECD_education_stats.csv"
education_data_df = pd.read_csv(csv_file)
education_data_df.head()

,GPS_CODE,GPS variables,LOCATION,Country,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,C002,Enrolment rate among 20-29 year-olds,AUS,Australia,1995,1995,PC,Percentage,0,Units,NaN,NaN,23.13,NaN,NaN
1,C002,Enrolment rate among 20-29 year-olds,AUS,Australia,2000,2000,PC,Percentage,0,Units,NaN,NaN,28.23,NaN,NaN
2,C002,Enrolment rate among 20-29 year-olds,AUS,Australia,2005,2005,PC,Percentage,0,Units,NaN,NaN,33.20,NaN,NaN
3,C002,Enrolment rate among 20-29 year-olds,AUS,Australia,2006,2006,PC,Percentage,0,Units,NaN,NaN,33.18,NaN,NaN
4,C002,Enrolment rate among 20-29 year-olds,AUS,Australia,2007,2007,PC,Percentage,0,Units,NaN,NaN,33.07,NaN,NaN


In [3]:
# Select columns
oecd_ed_df = education_data_df[['GPS variables', 'LOCATION', 'Country','TIME','Value', 'Unit']]
oecd_ed_df.head()

,GPS variables,LOCATION,Country,TIME,Value,Unit
0,Enrolment rate among 20-29 year-olds,AUS,Australia,1995,23.13,Percentage
1,Enrolment rate among 20-29 year-olds,AUS,Australia,2000,28.23,Percentage
2,Enrolment rate among 20-29 year-olds,AUS,Australia,2005,33.20,Percentage
3,Enrolment rate among 20-29 year-olds,AUS,Australia,2006,33.18,Percentage
4,Enrolment rate among 20-29 year-olds,AUS,Australia,2007,33.07,Percentage


In [4]:
# Check data types
oecd_ed_df.dtypes

# Check for NA values
oecd_ed_df.count() # Around 1200 values missing, but not dropping rows

GPS variables    4725
LOCATION         4725
Country          4725
TIME             4725
Value            3511
Unit             4725
dtype: int64

In [5]:
# Fix spelling errors (Enrolment >> Enrollment)
oecd_ed_df['GPS variables'] = oecd_ed_df['GPS variables'].replace(
    {'Enrolment rate among 20-29 year-olds':'Enrollment rate among 20-29 year-olds', 
     'Enrolment rate among 15-19 year-olds': 'Enrollment rate among 15-19 year-olds'
    })
oecd_ed_df.head()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,GPS variables,LOCATION,Country,TIME,Value,Unit
0,Enrollment rate among 20-29 year-olds,AUS,Australia,1995,23.13,Percentage
1,Enrollment rate among 20-29 year-olds,AUS,Australia,2000,28.23,Percentage
2,Enrollment rate among 20-29 year-olds,AUS,Australia,2005,33.20,Percentage
3,Enrollment rate among 20-29 year-olds,AUS,Australia,2006,33.18,Percentage
4,Enrollment rate among 20-29 year-olds,AUS,Australia,2007,33.07,Percentage


In [6]:
# Rename columns
oecd_ed_df = oecd_ed_df.rename(
    columns={'GPS variables':'Variables',
             'LOCATION':'Country Code',
             'TIME':'Year',
              })

oecd_ed_df.head()

,Variables,Country Code,Country,Year,Value,Unit
0,Enrollment rate among 20-29 year-olds,AUS,Australia,1995,23.13,Percentage
1,Enrollment rate among 20-29 year-olds,AUS,Australia,2000,28.23,Percentage
2,Enrollment rate among 20-29 year-olds,AUS,Australia,2005,33.20,Percentage
3,Enrollment rate among 20-29 year-olds,AUS,Australia,2006,33.18,Percentage
4,Enrollment rate among 20-29 year-olds,AUS,Australia,2007,33.07,Percentage


In [7]:
# Extract 2012 data
oecd_ed_2012=oecd_ed_df.loc[oecd_ed_df["Year"]==2012,:]

oecd_ed_2012.head()

,Variables,Country Code,Country,Year,Value,Unit
9,Enrollment rate among 20-29 year-olds,AUS,Australia,2012,34.52,Percentage
19,Enrollment rate among 20-29 year-olds,AUT,Austria,2012,25.85,Percentage
29,Enrollment rate among 20-29 year-olds,BEL,Belgium,2012,32.62,Percentage
39,Enrollment rate among 20-29 year-olds,CAN,Canada,2012,NaN,Percentage
49,Enrollment rate among 20-29 year-olds,CZE,Czech Republic,2012,25.98,Percentage


In [8]:
oecd_ed_2012=oecd_ed_2012.reset_index(drop=True)
oecd_ed_2012.head()

,Variables,Country Code,Country,Year,Value,Unit
0,Enrollment rate among 20-29 year-olds,AUS,Australia,2012,34.52,Percentage
1,Enrollment rate among 20-29 year-olds,AUT,Austria,2012,25.85,Percentage
2,Enrollment rate among 20-29 year-olds,BEL,Belgium,2012,32.62,Percentage
3,Enrollment rate among 20-29 year-olds,CAN,Canada,2012,NaN,Percentage
4,Enrollment rate among 20-29 year-olds,CZE,Czech Republic,2012,25.98,Percentage


In [9]:
# Create a dataframe for Enrollment rate among 20-29 year-olds
oecd_enroll_20=oecd_ed_2012.loc[oecd_ed_2012["Variables"]=="Enrollment rate among 20-29 year-olds",:]

# Change value column name to variable name and drop variable column
oecd_enroll_20=oecd_enroll_20.rename(columns={"Value": "Enrollment Rate Age 20-29"})
oecd_enroll_20=oecd_enroll_20.drop(['Variables', 'Unit'], axis=1)

oecd_enroll_20.head()

,Country Code,Country,Year,Enrollment Rate Age 20-29
0,AUS,Australia,2012,34.52
1,AUT,Austria,2012,25.85
2,BEL,Belgium,2012,32.62
3,CAN,Canada,2012,NaN
4,CZE,Czech Republic,2012,25.98


In [10]:
# Create dataframe for enrollment rate among 15-19 year-olds
oecd_enroll_15=oecd_ed_2012.loc[oecd_ed_2012["Variables"]=="Enrollment rate among 15-19 year-olds",:]

# Change value column name to variable name and drop variable and unit columns
oecd_enroll_15=oecd_enroll_15.rename(columns={"Value": "Enrollment Rate Age 15-19"})
oecd_enroll_15=oecd_enroll_15.drop(['Variables', 'Unit', 'Country', 'Year'], axis=1)

oecd_enroll_15.head()

,Country Code,Enrollment Rate Age 15-19
45,AUS,86.52
46,AUT,78.85
47,BEL,93.96
48,CAN,NaN
49,CZE,90.17


In [11]:
# Create dataframe for expected years of education for women
oecd_women_years=oecd_ed_2012.loc[oecd_ed_2012["Variables"]=="Expected number of years in education for women (All levels combined, Full-time), from age 5 through age 39",:]

# Change value column name to variable name and drop variable and unit columns
oecd_women_years=oecd_women_years.rename(columns={"Value": "Expected Education For Women (Years)"})
oecd_women_years=oecd_women_years.drop(['Variables', 'Unit', 'Country', 'Year'], axis=1)

oecd_women_years.head()

,Country Code,Expected Education For Women (Years)
585,AUS,16.21
586,AUT,17.28
587,BEL,16.57
588,CAN,15.85
589,CZE,17.94


In [12]:
# Create dataframe for expected years of education for women
oecd_men_years=oecd_ed_2012.loc[oecd_ed_2012["Variables"]=="Expected number of years in education for men (All levels combined, Full-time), from age 5 through age 39",:]

# Change value column name to variable name and drop variable and unit columns
oecd_men_years=oecd_men_years.rename(columns={"Value": "Expected Education For Men (Years)"})
oecd_men_years=oecd_men_years.drop(['Variables', 'Unit', 'Country', 'Year'], axis=1)

oecd_men_years.head()

,Country Code,Expected Education For Men (Years)
540,AUS,15.52
541,AUT,16.75
542,BEL,16.08
543,CAN,15.15
544,CZE,17.12


In [13]:
# Merge the four formatted dataframes together, set index
oecd_ed_stats_2012 = oecd_enroll_20.merge(oecd_enroll_15, on='Country Code').merge(oecd_women_years, on='Country Code').merge(oecd_men_years, on='Country Code')
oecd_ed_stats_2012.set_index("Country Code", inplace=True)

oecd_ed_stats_2012.head()

,Country,Year,Enrollment Rate Age 20-29,Enrollment Rate Age 15-19,Expected Education For Women (Years),Expected Education For Men (Years)
Country Code,,,,,,
AUS,Australia,2012,34.52,86.52,16.21,15.52
AUT,Austria,2012,25.85,78.85,17.28,16.75
BEL,Belgium,2012,32.62,93.96,16.57,16.08
CAN,Canada,2012,NaN,NaN,15.85,15.15
CZE,Czech Republic,2012,25.98,90.17,17.94,17.12


In [14]:
# Connect to local database
#database_path = "etl"
#engine = create_engine(f"sqlite:///{database_path}")

In [ ]:
# Check for tables
#engine.table_names()

In [ ]:
# Load converted OECD CSV table into database
#oecd_ed_stats_2012.to_sql(name='oecd_education_data', con=engine, if_exists='replace', index=True)

In [ ]:
# Confirm data has been loaded
#pd.read_sql_query('select * from oecd_education_data', con=engine).head(10)

In [ ]:
# Check for table names
#engine.table_names()

In [15]:
# Create a reference the CSV file desired
csv_path = "Resources/WorldBankData.csv"

# Read the CSV into a Pandas DataFrame
wbd_df = pd.read_csv(csv_path)

# Print the first five rows of data to the screen
wbd_df.head()

,Country Name,Country Code,Series Name,Series Code,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Australia,AUS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,16.1804,15.7206,15.2608,14.801,14.4212,14.0414,13.6616,13.2818,12.902,..
1,Australia,AUS,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,2.295077016,2.201174832,2.280410601,2.258752004,2.282509346,2.220048428,2.372696749,2.430579825,2.766645923,..
2,Australia,AUS,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,..,..,..,3.140243902,..,3.140243902,..,..,..,..
3,Australia,AUS,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,99.1,99.1,99.1,..,99.3,99.7,99.7,..,..,..
4,Australia,AUS,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,18.20018196,17.74084505,17.53887776,17.07290468,16.09583317,15.38876602,..,..,..,..


In [16]:
#del wbd_df ["Series Code"]
del wbd_df ["2009 [YR2009]"]
del wbd_df ["2010 [YR2010]"]
del wbd_df ["2011 [YR2011]"]
del wbd_df ["2013 [YR2013]"]
del wbd_df ["2014 [YR2014]"]
del wbd_df ["2015 [YR2015]"]
del wbd_df ["2016 [YR2016]"]
del wbd_df ["2017 [YR2017]"]
del wbd_df ["2018 [YR2018]"]

wbd_df.head()

,Country Name,Country Code,Series Name,Series Code,2012 [YR2012]
0,Australia,AUS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,14.801
1,Australia,AUS,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,2.258752004
2,Australia,AUS,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,3.140243902
3,Australia,AUS,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,..
4,Australia,AUS,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,17.07290468


In [17]:
wbd_df = wbd_df.rename(columns={'Country Name': 'Country_Name', 'Country Code': 'Country_Code', 'Series Name': 'Series_Name', 'Series Code': 'Series_Code'})



wbd_df.head()

,Country_Name,Country_Code,Series_Name,Series_Code,2012 [YR2012]
0,Australia,AUS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,14.801
1,Australia,AUS,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,2.258752004
2,Australia,AUS,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,3.140243902
3,Australia,AUS,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,..
4,Australia,AUS,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,17.07290468


In [18]:
#wbd_3_df = wbd_df.loc[wbd_df["Series_Code"]=="SH.STA.BRTC.ZS",:]
wbd_6_df = wbd_df.loc[wbd_df["Series_Code"]=="FS.AST.DOMS.GD.ZS",:]
#wbd_10_df = wbd_df.loc[wbd_df["Series_Code"]=="DT.DOD.DECT.CD",:]
wbd_11_df = wbd_df.loc[wbd_df["Series_Code"]=="SP.DYN.TFRT.IN",:]
wbd_16_df = wbd_df.loc[wbd_df["Series_Code"]=="NY.GNP.PCAP.CD",:]
wbd_17_df = wbd_df.loc[wbd_df["Series_Code"]=="NY.GNP.PCAP.PP.CD",:]
wbd_31_df = wbd_df.loc[wbd_df["Series_Code"]=="SH.DYN.MORT",:]
#wbd_40_df = wbd_df.loc[wbd_df["Series_Code"]=="SI.POV.NAHC",:]
wbd_52_df = wbd_df.loc[wbd_df["Series_Code"]=="IC.REG.DURS",:]

In [19]:
wbd_merge_1 = [wbd_6_df, wbd_11_df]

In [20]:
wbd_merge_1 = reduce(lambda left,right: pd.merge(left,right,on=["Country_Name","Country_Code"]), wbd_merge_1)

wbd_merge_1.head()

,Country_Name,Country_Code,Series_Name_x,Series_Code_x,2012 [YR2012]_x,Series_Name_y,Series_Code_y,2012 [YR2012]_y
0,Australia,AUS,Domestic credit provided by financial sector (...,FS.AST.DOMS.GD.ZS,150.1579391,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1.92
1,France,FRA,Domestic credit provided by financial sector (...,FS.AST.DOMS.GD.ZS,147.6736927,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,2.01
2,Germany,DEU,Domestic credit provided by financial sector (...,FS.AST.DOMS.GD.ZS,156.9249454,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1.41
3,Italy,ITA,Domestic credit provided by financial sector (...,FS.AST.DOMS.GD.ZS,177.3916251,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1.43
4,Japan,JPN,Domestic credit provided by financial sector (...,FS.AST.DOMS.GD.ZS,329.7278431,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1.41


In [21]:
del wbd_merge_1 ["Series_Code_y"]
del wbd_merge_1 ["Series_Code_x"]
del wbd_merge_1 ["Series_Name_y"]
del wbd_merge_1 ["Series_Name_x"]

In [22]:
wbd_merge_1 = wbd_merge_1.rename(columns={"2012 [YR2012]_x": 'Domestic Credit', "2012 [YR2012]_y": "Fertility Rate"})
wbd_merge_1.head()

,Country_Name,Country_Code,Domestic Credit,Fertility Rate
0,Australia,AUS,150.1579391,1.92
1,France,FRA,147.6736927,2.01
2,Germany,DEU,156.9249454,1.41
3,Italy,ITA,177.3916251,1.43
4,Japan,JPN,329.7278431,1.41


In [23]:
wbd_merge_2 = [wbd_16_df, wbd_17_df]

In [24]:
wbd_merge_2 = reduce(lambda left,right: pd.merge(left,right,on=["Country_Name","Country_Code"]), wbd_merge_2)

wbd_merge_2.head()

,Country_Name,Country_Code,Series_Name_x,Series_Code_x,2012 [YR2012]_x,Series_Name_y,Series_Code_y,2012 [YR2012]_y
0,Australia,AUS,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,60090,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,41450
1,France,FRA,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,43380,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,38500
2,Germany,DEU,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,46710,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,44590
3,Italy,ITA,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,36000,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,36190
4,Japan,JPN,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,49480,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,38190


In [25]:
wbd_merge_2 = wbd_merge_2.rename(columns={"2012 [YR2012]_x": 'GNI Per Capita', "2012 [YR2012]_y": "GNI Per Capita, PPP"})
wbd_merge_2.head()

,Country_Name,Country_Code,Series_Name_x,Series_Code_x,GNI Per Capita,Series_Name_y,Series_Code_y,"GNI Per Capita, PPP"
0,Australia,AUS,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,60090,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,41450
1,France,FRA,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,43380,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,38500
2,Germany,DEU,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,46710,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,44590
3,Italy,ITA,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,36000,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,36190
4,Japan,JPN,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,49480,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,38190


In [26]:
del wbd_merge_2 ["Series_Code_y"]
del wbd_merge_2 ["Series_Code_x"]
del wbd_merge_2 ["Series_Name_y"]
del wbd_merge_2 ["Series_Name_x"]
wbd_merge_2.head()

,Country_Name,Country_Code,GNI Per Capita,"GNI Per Capita, PPP"
0,Australia,AUS,60090,41450
1,France,FRA,43380,38500
2,Germany,DEU,46710,44590
3,Italy,ITA,36000,36190
4,Japan,JPN,49480,38190


In [27]:
wbd_merge_3 = [wbd_31_df, wbd_52_df]

In [28]:
wbd_merge_3 = reduce(lambda left,right: pd.merge(left,right,on=["Country_Name","Country_Code"]), wbd_merge_3)
wbd_merge_3.head()

,Country_Name,Country_Code,Series_Name_x,Series_Code_x,2012 [YR2012]_x,Series_Name_y,Series_Code_y,2012 [YR2012]_y
0,Australia,AUS,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,4.3,Time required to start a business (days),IC.REG.DURS,2.5
1,France,FRA,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,4.1,Time required to start a business (days),IC.REG.DURS,6.5
2,Germany,DEU,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,4,Time required to start a business (days),IC.REG.DURS,14.5
3,Italy,ITA,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,3.8,Time required to start a business (days),IC.REG.DURS,7.5
4,Japan,JPN,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,3,Time required to start a business (days),IC.REG.DURS,..


In [29]:
wbd_merge_3 = wbd_merge_3.rename(columns={"2012 [YR2012]_x": 'Mortality Rate Under 5', "2012 [YR2012]_y": "Days to Start a Business"})
wbd_merge_3.head()

,Country_Name,Country_Code,Series_Name_x,Series_Code_x,Mortality Rate Under 5,Series_Name_y,Series_Code_y,Days to Start a Business
0,Australia,AUS,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,4.3,Time required to start a business (days),IC.REG.DURS,2.5
1,France,FRA,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,4.1,Time required to start a business (days),IC.REG.DURS,6.5
2,Germany,DEU,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,4,Time required to start a business (days),IC.REG.DURS,14.5
3,Italy,ITA,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,3.8,Time required to start a business (days),IC.REG.DURS,7.5
4,Japan,JPN,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,3,Time required to start a business (days),IC.REG.DURS,..


In [30]:
del wbd_merge_3 ["Series_Code_y"]
del wbd_merge_3 ["Series_Code_x"]
del wbd_merge_3 ["Series_Name_y"]
del wbd_merge_3 ["Series_Name_x"]
wbd_merge_3.head()

,Country_Name,Country_Code,Mortality Rate Under 5,Days to Start a Business
0,Australia,AUS,4.3,2.5
1,France,FRA,4.1,6.5
2,Germany,DEU,4,14.5
3,Italy,ITA,3.8,7.5
4,Japan,JPN,3,..


In [31]:
wbd_merge_all = [wbd_merge_1, wbd_merge_2, wbd_merge_3]

In [32]:
wbd_merge_all = reduce(lambda left,right: pd.merge(left,right,on=["Country_Name","Country_Code"]), wbd_merge_all)
wbd_merge_all.head()

,Country_Name,Country_Code,Domestic Credit,Fertility Rate,GNI Per Capita,"GNI Per Capita, PPP",Mortality Rate Under 5,Days to Start a Business
0,Australia,AUS,150.1579391,1.92,60090,41450,4.3,2.5
1,France,FRA,147.6736927,2.01,43380,38500,4.1,6.5
2,Germany,DEU,156.9249454,1.41,46710,44590,4,14.5
3,Italy,ITA,177.3916251,1.43,36000,36190,3.8,7.5
4,Japan,JPN,329.7278431,1.41,49480,38190,3,..


In [33]:
# Connect to local database
database_path = "etl"
engine = create_engine(f"sqlite:///{database_path}")

In [34]:
# Check for tables
engine.table_names()

[]

In [35]:
# Load converted OECD CSV table into database
oecd_ed_stats_2012.to_sql(name='oecd_education_data', con=engine, if_exists='replace', index=True)

In [36]:
# Confirm data has been loaded
pd.read_sql_query('select * from oecd_education_data', con=engine).head(10)

,Country Code,Country,Year,Enrollment Rate Age 20-29,Enrollment Rate Age 15-19,Expected Education For Women (Years),Expected Education For Men (Years)
0,AUS,Australia,2012,34.52,86.52,16.21,15.52
1,AUT,Austria,2012,25.85,78.85,17.28,16.75
2,BEL,Belgium,2012,32.62,93.96,16.57,16.08
3,CAN,Canada,2012,NaN,NaN,15.85,15.15
4,CZE,Czech Republic,2012,25.98,90.17,17.94,17.12
5,DNK,Denmark,2012,43.25,87.38,19.25,18.50
6,FIN,Finland,2012,41.76,85.92,18.54,17.54
7,FRA,France,2012,20.93,83.63,16.77,16.13
8,DEU,Germany,2012,33.07,89.68,17.62,17.89
9,GRC,Greece,2012,41.50,85.33,18.54,18.21


In [37]:
# Load converted World Bank CSV table into database
wbd_merge_all.to_sql(name='world_bank_data', con=engine, if_exists='replace', index=True)

In [38]:
# Confirm data has been loaded
pd.read_sql_query('select * from world_bank_data', con=engine).head(10)

,index,Country_Name,Country_Code,Domestic Credit,Fertility Rate,GNI Per Capita,"GNI Per Capita, PPP",Mortality Rate Under 5,Days to Start a Business
0,0,Australia,AUS,150.1579391,1.92,60090,41450,4.3,2.5
1,1,France,FRA,147.6736927,2.01,43380,38500,4.1,6.5
2,2,Germany,DEU,156.9249454,1.41,46710,44590,4,14.5
3,3,Italy,ITA,177.3916251,1.43,36000,36190,3.8,7.5
4,4,Japan,JPN,329.7278431,1.41,49480,38190,3,..
5,5,"Korea, Rep.",KOR,155.8450252,1.297,24550,32430,3.8,4
6,6,Mexico,MEX,45.63480336,2.295,9750,16340,16.4,..
7,7,Netherlands,NLD,231.3754468,1.72,52930,48080,4.2,5
8,8,Spain,ESP,245.5703178,1.32,29760,31760,3.6,30
9,9,Switzerland,CHE,182.2955505,1.52,84880,59140,4.4,18


In [39]:
# Check for table names
engine.table_names()

['oecd_education_data', 'world_bank_data']